In [27]:
import io
import zipfile

import requests
import pandas as pd

In [11]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    style="{",
    format="{levelname}:{module}.{funcName}:{lineno}:{message}"
)

In [12]:
logger = logging.getLogger()

In [14]:
import ausweather.bom

from ausweather.bom import resolve_ncc_obs_code, fetch_bom_c_values

DEBUG:pyplot.switch_backend:225:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:pyplot.switch_backend:225:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [15]:
df = ausweather.bom.fetch_bom_c_values("daily_rain", 23000, radius_km=0)

DEBUG:bom.fetch_bom_c_values:101:Using resolved ncc_obs_code 136
DEBUG:connectionpool._new_conn:225:Starting new HTTP connection (1): www.bom.gov.au:80
DEBUG:connectionpool._make_request:437:http://www.bom.gov.au:80 "GET /jsp/ncc/cdio/weatherStationDirectory/d?p_display_type=ajaxStnListing&p_nccObsCode=136&p_stnNum=23000&p_radius=0 HTTP/1.1" 200 813


In [16]:
df

,Km,Station,Name,Lat,Lon,Elev,First,Last,Years,%,c
0,0.0,23000,Adelaide (West Terrace / Ngayirdapira),-34.9257,138.5832,29.32,1839 Jan,2020 Apr,142.9,79,-105818465


In [19]:
def fetch_bom_weather_data_complete(ncc_obs_code, station_code):
    logger.debug(f"Passed parameter ncc_obs_code {ncc_obs_code}")
    var = resolve_ncc_obs_code(ncc_obs_code)
    logger.debug(f"Resolved ncc_obs_code as variable {var}")
    c_table = fetch_bom_c_values(var["ncc_obs_code"], station_code, radius_km=0)
    p_c = c_table.c.iloc[0]
    url = (
        f"http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?"
        f"p_display_type={var['interval']}ZippedDataFile&"
        f"p_stn_num={station_code:06.0f}&p_nccObsCode={var['ncc_obs_code']:.0f}"
        f"&p_c={p_c:.0f}"
    )
    r = requests.get(url)
    buffer = io.BytesIO(r.content)
    

In [20]:
x = fetch_bom_weather_data_complete("monthly_rain", 23000)

DEBUG:<ipython-input-19-d909c9215d3d>.fetch_bom_weather_data:2:Passed parameter ncc_obs_code monthly_rain
DEBUG:<ipython-input-19-d909c9215d3d>.fetch_bom_weather_data:4:Resolved ncc_obs_code as variable {'name': 'Rainfall - monthly total', 'ncc_obs_code': 139, 'interval': 'monthly', 'aliases': ['monthly_rain']}
DEBUG:bom.fetch_bom_c_values:101:Using resolved ncc_obs_code 139
DEBUG:connectionpool._new_conn:225:Starting new HTTP connection (1): www.bom.gov.au:80
DEBUG:connectionpool._make_request:437:http://www.bom.gov.au:80 "GET /jsp/ncc/cdio/weatherStationDirectory/d?p_display_type=ajaxStnListing&p_nccObsCode=139&p_stnNum=23000&p_radius=0 HTTP/1.1" 200 813
DEBUG:connectionpool._new_conn:225:Starting new HTTP connection (1): www.bom.gov.au:80
DEBUG:connectionpool._make_request:437:http://www.bom.gov.au:80 "GET /jsp/ncc/cdio/weatherData/av?p_display_type=monthlyZippedDataFile&p_stn_num=023000&p_nccObsCode=139&p_c=-105819125 HTTP/1.1" 200 15751


In [28]:
zfile = zipfile.ZipFile(x)
filenames = zfile.namelist()
data_filenames = [fn for fn in filenames if fn.contains("_Data")]
dfs = {
    fn.split(".")[0]: pd.read_csv(zfile.open(fn), compression="zip"
}

In [29]:
zfile.namelist()

['IDCJAC0001_023000_Data1.csv',
 'IDCJAC0001_023000_Data12.csv',
 'IDCJAC0001_023000_Note.txt']

In [35]:
for i, line in enumerate(zfile.read('IDCJAC0001_023000_Data1.csv').decode("ascii").splitlines()):
    print(line)
    if i == 5:
        break

Product code,Station number,Year,Month,Monthly Precipitation Total (millimetres),Quality
IDCJAC0001,023000,1839,01,11.5,Y
IDCJAC0001,023000,1839,02,11.4,Y
IDCJAC0001,023000,1839,03,20.8,Y
IDCJAC0001,023000,1839,04,10.5,Y
IDCJAC0001,023000,1839,05,4.8,Y
